## Evaluation Dataset Preparation

In [1]:
import pandas as pd
import numpy as np
import random
import ast
import nlpaug.augmenter.word as naw
from math import sqrt
from sklearn.model_selection import train_test_split

2023-03-06 11:19:23.815825: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-03-06 11:19:25.175774: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-06 11:19:25.175884: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-06 11:19:25.175895: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
df = pd.read_csv('../data/lyrics.csv')

In [3]:
df.head()

,lyrics,genre,id
0,"['So long', 'My sweet love', 'I miss you', ""Mo...",pop,0
1,"[""It won't be long before I'm leaving here"", ""...",pop,1
2,"['Are we crazy?', 'Living our lives through a ...",pop,2
3,"['When did it go wrong, I will never know', 'I...",pop,3
4,"[""I've waited all this time"", 'Counting minute...",pop,4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36000 entries, 0 to 35999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lyrics  36000 non-null  object
 1   genre   36000 non-null  object
 2   id      36000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 843.9+ KB


In [5]:
df['lyrics'] = df['lyrics'].apply(ast.literal_eval)

In [6]:
df0 = df.copy()

In [7]:
df = df.groupby("genre").sample(n=50, random_state=42).reset_index()

In [8]:
eg_lyrics = random.choice(df['lyrics'])
eg_lyrics

['I get that paper, I do it major',
 "I'm stayin sucker-free and, I'm shakin haters ",
 "I ain't really tryin to hurt nobody ",
 "I ain't really tryin to hurt nobody ",
 "I ain't really tryin to hurt nobody",
 'Is somebody gonna get hurt up in here? Probably',
 'My money get jealous, I keep the dough in check',
 "The checks bounce, give me cash and give me mo' interest",
 'Rick Rock, now I get the dough goin west',
 "20 thou' on the ground, what'chu wanna bet?",
 'If I lose I roll another bet to even up',
 "Keep lookin for the dough like I ain't seen enough",
 'Fully equipped the whip, that would be a plus',
 "Hop off the you-know-what and I'ma leave it up",
 " The dough on the Lambo'",
 "Don't try to be a Rambo or be an example",
 "Passport paper, ain't a place that I can't go",
 "Haters gettin sick 'bout to Earl, like Kimbo ",
 'Go ahead man, smack yourself silly',
 'Rubberbands never fit when I package my scrilly',
 "Overseas but I'm still the man back in my city",
 'Ugh, put the cr

In [9]:
aug = naw.ContextualWordEmbsAug(model_path='roberta-base', action="substitute")

In [10]:
def misalign_lyrics(lyrics, repetitions, repeat_verse):
    verses2repeat = random.sample(range(0, len(lyrics)), repetitions)
    misaligned_lyrics = []
    for idx, verse in enumerate(lyrics):
        if idx in verses2repeat:
            misaligned_lyrics.extend([verse] * repeat_verse)
        else:
            misaligned_lyrics.append(verse)
    return misaligned_lyrics

In [11]:
evaluation_ids = list(df['id'])

all_lyrics = df0.drop(index=evaluation_ids)['lyrics']

In [12]:
eg_verse = random.choice(eg_lyrics)
print('Original verse:')
print(eg_verse)

print('Augmented:')
aug.augment(eg_verse)

Original verse:
Yup! That mean minus a couple
Augmented:


['Yup! That puts minus 1 year']

In [13]:
eg_lyrics = random.choice(df['lyrics'])

eg_aug_lyrics = aug.augment(eg_lyrics)

eg_aug_misl_lyrics = misalign_lyrics(eg_aug_lyrics, 
                                     repetitions=round(sqrt(len(eg_lyrics))), 
                                     repeat_verse=2)

eg_fake_lyrics = random.choice(all_lyrics)

print('ORIGINAL lyrics:')
print(eg_lyrics)
print()
print('AUGMENTED lyrics:')
print('Aligned:')
print(eg_aug_lyrics)
print()
print('Misaligned:')
print(eg_aug_misl_lyrics)
print()
print('RANDOM lyrics:')
print(eg_fake_lyrics)

ORIGINAL lyrics:
['When I was a little girl I never was a princess type', "I didn't do fairy tales or wish on stars at night", 'All the other girls are living in make believe', 'Searching for the one boy of their dreams', 'Ooh ooh ooh, ooh ooh ooh, ooh ooh ooh', 'Everywhere I go see a trail of broken hearts', 'And one by one I watch them fall apart', "I told myself I won't let that be me", "I'll take my time for ever after happily", "I'll save my love for love in a land called far away", "Until then well, love, your heart's gonna have to wait", 'When I find my love that I will love every day', "Ooh ooh, where hearts don't have to break", "And I'll stay in love in a land called far away", 'Ooh ooh ooh, ooh ooh ooh, ooh ooh ooh', 'Ooh ooh ooh, ooh ooh ooh, ooh ooh ooh', "Run along little boy, you're just distracting me", "I've got things to learn and places I should be", 'Though I appreciate the way you feel for me', "Truth is I'm miles away from feeling that kind of thing", "I'll save m

In [14]:
aug_lyrics = df['lyrics'].apply(lambda x: aug.augment(x))


misl_aug_lyrics = [misalign_lyrics(aug_lyr, repetitions=round(sqrt(
    len(aug_lyr))), repeat_verse=random.choice([2,3])) for aug_lyr in aug_lyrics]

                                   
df['similar_lyrics'] = misl_aug_lyrics
df['random_lyrics'] = random.sample(list(all_lyrics), len(df['lyrics']))

In [15]:
df.head()

,index,lyrics,genre,id,similar_lyrics,random_lyrics
0,1935,"[Well, I've been waitin' ever since eight, Gue...",pop,1935,"[Well, he liked waitin' ever minute..., Guess ...","[Little girls, listen closely, 'Cause no one t..."
1,6494,"[This kind of love, Is more than a lifeline, F...",pop,6494,"[This to the love, Is more we a this, For a ca...","[What you're gonna do, You ain't no friend of ..."
2,1720,"[I could tell by the look in her eyes, Maybe I...",pop,1720,"[I barely can by the fail in her eyes, Maybe I...","[Swag central, See what we have here is, An op..."
3,9120,"[Don't Tell me you love me when I don't, You k...",pop,9120,"[Don't call me you love me about this miss, Yo...","[Baby if you really love me, You had better un..."
4,360,"[Who's the man? who's the boss?, who's the nig...",pop,360,"[Who's the &? why came boss?, who's the<unk> t...","[Every day I see your face, I wish that I'd st..."


In [16]:
df.to_csv('../data/evaluation/lyrics_eval.csv', index=False)